# Model CNN pour classification

In [2]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt

## Dataset

In [3]:
img_height = 32
img_width = 32
batch_size = 32

train_data = tf.keras.utils.image_dataset_from_directory(
    '../Data/gems_images/train',
    validation_split=0.2,          
    subset="training",
    seed=42,                       
    image_size=(img_height, img_width),
    batch_size=batch_size
)

validation_data = tf.keras.utils.image_dataset_from_directory(
    '../Data/gems_images/train',
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

test_data = tf.keras.utils.image_dataset_from_directory(
    '../Data/gems_images/test',
    image_size=(img_height, img_width),
    batch_size=batch_size
)


NotFoundError: Could not find directory ../Data/gems_images/train

## Data visualisation

In [ ]:
class_names = train_data.class_names
print("Class names:", class_names)
# Data visualisation
plt.figure(figsize=(7, 7))
for images, labels in train_data.take(1):
    for i in range(16):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

## creation du model

In [ ]:
num_classes = len(class_names)

modelCNN = tf.keras.Sequential([

    # Normalize
    tf.keras.layers.Rescaling(1./255),

    # --- Block 1 ---
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),

    # --- Block 2 ---
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.3),

    # --- Block 3 ---
    tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.4),

    # Flatten + Dense
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    # Output layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

In [ ]:
modelCNN.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = modelCNN.fit(
    train_data,
    validation_data=validation_data,
    epochs=20
)

## Plot training curves

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 5))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend()
plt.title('Training vs Validation Loss')

plt.show()


In [ ]:
modelCNN.fit(test_data,verbose=2)

In [ ]:
modelCNN.evaluate(test_data,verbose=2)

In [ ]:
for images, labels in test_data.take(1):
    prediction = modelCNN.predict(images)
    predicted_labels = np.argmax(prediction, axis=1)
    for i in range(10):
        print(f"Image {i+1}: True label: {class_names[labels[i]]}, Predicted label: {class_names[predicted_labels[i]]}")